# How to decorate the run_step() method (and why)

The use of decorators is optional and intended to structure and make the `run_step()`method clearer and more compact.
In order to use the decorators you have to import them as follows:

In [1]:
import progressivis.core.decorators

or :

In [2]:
from progressivis.core.decorators import process_slot, run_if_any # , etc.

Currently there are two categories of decorators:

* Slot processing decorators [sp-decorators]
* Run condition decorators [rc-decorators]

The two categories are inseparable.
Of course you can develop run_step without decorators but if you choose to use the decorators, the `run_step()` method must be decorated by at least one sp-decorator followed by at least one rc-decorator

## Slot processing decorators

For now this category has only one decorator but it can be applied multiple times.


In [3]:
def process_slot(*names, reset_if=('update', 'delete'), reset_cb=None):
    pass

* `reset_if` indicates if resetting is required and in which case. By default the slot is reseted if deletions or modifications occurred on the input data. Possible values are:
  * `reset_if='update'` slot is reseted only if modifications occurred
  * `reset_if='delete'` slot is reseted only if deletions occurred
  * `reset_if='False'` slot is NOT reseted in any case
* `reset_cb` is pertinent only when `reset_if` is not **False**. For now `reset_cb` can contain a method name (i.e. a string) to be called after the slot has been reseted. The method must not have arguments (except `self`)

We will apply `process_slot()` once for all slots requiring the same treatment :

In [4]:
from progressivis.table.module import TableModule
from progressivis.core.slot import SlotDescriptor
from progressivis.table.table import Table
from progressivis.core.decorators import *

class FooModule(TableModule):
    inputs = [SlotDescriptor('a', type=Table, required=True),
              SlotDescriptor('b', type=Table, required=True),
              SlotDescriptor('c', type=Table, required=True),
              SlotDescriptor('d', type=Table, required=True),
    ]

    @process_slot("a", "b", "c", "d", reset_if=False)    
    @run_if_any # mandatory run condition decorator, explained below
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

We can  apply process_slot() many times when the treatments on slots differ:

In [5]:
class FooModule(TableModule):
    inputs = [SlotDescriptor('a', type=Table, required=True),
              SlotDescriptor('b', type=Table, required=True),
              SlotDescriptor('c', type=Table, required=True),
              SlotDescriptor('d', type=Table, required=True),
    ]

    def reset(self):
        pass # do some reset related treatments
    
    @process_slot("a", "b", reset_cb='reset') # by default reset_if=('update', 'delete')
    @process_slot("c", reset_if='update')
    @process_slot("d", reset_if=False)
    @run_if_any # mandatory run condition decorator, explained below
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

## Run condition decorators

These decorators define the conditions that allow the execution of the decorated run_step () method.

They are :

* @run_if_any with possible extension @and_any
* @run_if_all with possible extension @or_all
* @run_always

### The @run_if_any decorator

Allows execution of the decorated run_step() method if and only if at least one entry contains new data. It can be used with or without arguments which are slot names. When called without arguments it applies to all entry slots:

In [6]:
    # @run_if_any without arguments
    @process_slot("a", "b", "c", "d") 
    @run_if_any # run if at least one among "a", "b", "c", "d" slots contains new data
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

In [7]:
    # @run_if_any with arguments    
    @process_slot("a", "b", "c", "d") 
    @run_if_any("b", "d") # run if at least one between b" and "d" slots contains new data
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

### The @and_any extension decorator

It makes possible nested conditions (2 levels) in the form :
`(a | b | ...) & (x | y | ...) & ...`

In [8]:
    # (a|c) & (b|d)
    @process_slot("a", "b", "c", "d") 
    @run_if_any("a", "c")
    @and_any("b", "d") 
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

### The @run_if_all decorator

Allows execution of the decorated run_step() method if and only if all entries contain new data. It can be used with or without arguments which are slot names. When called without arguments it applies to all entry slots:

In [9]:
    # @run_if_all without arguments
    @process_slot("a", "b", "c", "d") 
    @run_if_all # all "a", "b", "c", "d" slots contains new data
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

In [10]:
    # @run_if_all with arguments
    @process_slot("a", "b", "c", "d") 
    @run_if_all("b", "d") # run if both b" and "d" slots contains new data
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

### The @or_all extension decorator

It makes possible nested conditions (2 levels) in the form :
`(a & b & ...) | (x & y & ...) | ...`

In [11]:
    # (a&c) | (b&d)
    @process_slot("a", "b", "c", "d") 
    @run_if_all("a", "c")
    @or_all("b", "d") 
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something

### The @run_always decorator

Allows the execution of the decorated run_step() method **always**.


In [12]:
    @process_slot("a", "b", "c", "d") 
    @run_always
    def run_step(self, run_number, step_size, howlong):
        with self.context as ctx:
            pass # do something